###### 겨리 - 2020/01/15
### **고윳값과 고유벡터(미완료)**

### 1. 고윳값과 고유벡터를 활용한 벡터공간의 변환
#### Ax = b와 같은 선형 변환을 데카르트 좌표계(좌표평면 등)가 아닌 벡터의 상수배로 표현할 수 있도록 변환하는 것
##### 고유벡터(eigen vector) : 변환에 의해 구성된 새로운 공간(고유공간)의 축 ->
##### 고윳값(eigen value) : 벡터의 상수배인 새로운 축을 위한 값
###### *엄밀한 정의는 아님(https://ko.wikipedia.org/wiki/%EA%B3%A0%EC%9C%B3%EA%B0%92%EA%B3%BC_%EA%B3%A0%EC%9C%A0_%EB%B2%A1%ED%84%B0 참조)
##### 코드 설명
###### https://gyeo-ri.com/62

In [2]:
import numpy as np
import sympy as sym

##### 주요 외부 라이브러리
##### np.identity : 단위행렬
##### sym.symbols : 변수 객체를 지정
##### sym.solve : 방정식 풀이

In [3]:
A = np.array([[-5.,-4.,2.],[-2., -2.,2.],[4.,2.,2.]])
I = np.identity(3)

##### 가우스 소거법 형태로 행렬식을 푸는 np.linalg.det는 Symbol 객체를 입력하면 오류가 발생(det_gauss도 마찬가지)
##### 행렬식 노트에서 정의하였던 determinant 함수 또는 여인수 전개 기반 det_cofactor 사용(라이프니츠 공식 기반) 사용 가능
##### 여인수 전개를 사용했을 때 훨씬 효율적(7차 정방행렬 계산 시 0.4s vs 14s

In [4]:
l = sym.symbols('l') #람다 변수 객체 정의
matrix_cal = l*I - A #고윳값 계산을 위한 

In [9]:
equation = det_cofactor(matrix_cal) #특성방정식(가우스 소거법을 사용하는 np.linalg.det는 사용 불가)
equation

-16.0*l + (1.0*l + 5.0)*((1.0*l - 2.0)*(1.0*l + 2.0) - 4.0) + 40.0

#### 특성방정식 det(lI-A) 
##### (lI-A)x=O 에서 x가 자명해를 가지기 위해(역행렬의 이항이 가능하면 무수히 많은 해를 가짐)
##### 람다에 관현 방정식을 풀면 고윳값을 얻을 수 있음
##### 위 코드들을 바탕으로 구현한 eigen_value 출력 함수

In [302]:
def eigen_value(matrix):
    
    dim = len(matrix)
    I = np.identity(dim)
    
    l = sym.symbols('l') #람다 변수 객체 정의
    matrix_cal = l*I - A #고윳값 계산을 위한 연립방정식
    equation = det_cofactor(matrix_cal) #행렬식으로 변환(가우스 소거법을 사용하는 np.linalg.det는 사용 불가)
    
    eigen_value = sym.solve(equation)
    
    return eigen_value

##### 실수 및 복소수인 고윳값을 모두 출력함

In [298]:
A = np.array([[-5.,-4.,2.],[-2., -2.,2.],[4.,2.,2.]])
eigen_value(A)

[-8.00000000000000, 0.0, 3.00000000000000]

In [341]:
def eigen_vector(matrix):
    dim = len(matrix)
    I = np.identity(dim)
    
    vector = []
    # 고윳값을 하나씩 추출하는 반복문을 아래 모든 구문에 적용
    eigenvalues = eigen_value(A)
    
    print("고윳값")
    print(eigenvalues)
    for l in eigenvalues :
        try :
            l = float(l)
        except : #복소수 계산기능을 구현하지 못함
            return print("복소수는 계산할 수 없습니다.")
        flag = 0
        matrix_cal = l*I - A
    

        #입력 행렬의 1차 검증 : 값이 모두 0인 열이 있는지 -> 모두 0인 열이 1열이라면, 이에 대응하는 v1(1,0,0) ... 가 생성
        for c in range(dim): #열
            if (matrix_cal[:,c] == np.zeros((dim,1),dtype=float)).all():

                v = np.zeros((dim,1))
                v[c] = 1
                vector.append(v)
                flag = 1



        if flag == 0: # 한 열이 모두 0이 아닌 행렬

            #행의 재배치(0)
            for row in range(dim):
                if (matrix_cal[row] == np.zeros((1,dim),dtype=float)).all():

                    #행을 중간에서 빼서 마지막에 넣는 알고리즘
                    matrix_cal = np.delete(matrix_cal.copy(),row,axis=0)
                    matrix_cal = np.concatenate((matrix_cal,np.zeros((1,dim))))




            #1. 가우스 소거법 : Leading 1(선행원소 1)을 찾기 위한 알고리즘
            row_count = 0 #연산 중인 열을 지시
            for col_count in range(dim-1): #선행 원소 아래의 값(열 하단)이 0이 될때까지 반복 수행
                comp_num = np.inf # 비교할 숫자의 기본값을 무한대로 지정(보다 작은 수)


                # 첫 열이 1이거나, 0이 아닌 수 중 가장 작은 행을 추출
                for i in range(row_count,dim-1):
                    if matrix_cal[i][col_count] != 0 : 
                        comp_num = matrix_cal[i][col_count] #절댓값이 0이 아닌 값을 위로
                        first_row = i #첫 행으로 사용할 행
                        
                        if matrix_cal[i][col_count] == 1.: #1을 찾은 경우 
                            comp_num = 1
                            break
                            
                    else : 
                        comp_num = 1 # 적당한 값이 없을 때는 상수배를 하지 않음

                matrix_cal[first_row]/=comp_num  #첫 행의 첫 열을 1로 변환
                
  
                
                if first_row != row_count :

                    matrix_cal[first_row], matrix_cal[row_count] = matrix_cal[row_count].copy(), matrix_cal[first_row].copy() #대상 인덱스와 첫 행을 교환

                    

                #상수배 후 덧셈연산
                for j in range(row_count+1,dim):
                    if matrix_cal[j][col_count] != 0:
                        con_no = matrix_cal[j][col_count]
                        matrix_cal[j] = matrix_cal[j] - con_no * matrix_cal[row_count]


                #이미 Leading 1을 찾은 행을 연산에서 제외
                row_count += 1

            
            #행의 재배치(0)
            for row in range(dim):
                if (matrix_cal[row] == np.zeros((1,dim),dtype=float)).all():

                    #행을 중간에서 빼서 마지막에 넣는 알고리즘
                    matrix_cal = np.delete(matrix_cal.copy(),row,axis=0)
                    matrix_cal = np.concatenate((matrix_cal,np.zeros((1,dim))))


            # 2. Back Substitution(가우스-조르당 소거법) : 기약행사다리꼴 형태로 변환
            for col in range(dim-1,0,-1):
                for row in range(col-1,-1,-1): #col/row를 파이썬 인덱스에 맞게 적용
                    const = matrix_cal[row][col] * -1 #constraint

                    matrix_cal[row] += const * matrix_cal[col]



            #벡터를 구하는 알고리즘
            
            
            if (matrix_cal[0] != np.zeros((1,dim))).all():
                
                for m in range(dim-1): #if 긍정조건은 벡터가 반복문마다 하나씩 리턴 / 벡터를 반복문 내부에서 생성
                    v = np.zeros((dim,1))
                    v[-1] = matrix_cal[0][-1] #첫줄 마지막 원소
                    v[m] =  - v[-1]/matrix_cal[0][m]
                    vector.append(v) 

            
            
            
            else :
                v = np.zeros((dim,1))
                for k in range(dim-1): 
                    v[-1] = 1.
                    x = matrix_cal[k][k].copy()
                    end = matrix_cal[k][-1].copy()
                    v[k] = (-1) * end/x
                vector.append(v) #부정조건은 하나의 벡터만 생성되므로 반복문 외부에서 생성
            
            
    return  vector

##### 1.중근을 포함하지 않는 4차 정방행렬로 테스트

In [344]:
A = np.array([[1.,0.,0.,0.],[-3., -2.,0.,0.],[2.,3.,-1.,0.],[-2.,3.,0.,3.]])
eigen_vector(A)

고윳값
[-2.00000000000000, -1.00000000000000, 1.00000000000000, 3.00000000000000]


[array([[-0.        ],
        [-1.66666667],
        [ 5.        ],
        [ 1.        ]]),
 array([[0.],
        [0.],
        [1.],
        [0.]]),
 array([[ 0.4],
        [-0.4],
        [-0.2],
        [ 1. ]]),
 array([[0.],
        [0.],
        [0.],
        [1.]])]

##### 2. 중근을 포함하는 3차 정방행렬 테스트

In [343]:
A = np.array([[0.,1.,1.],[1., 0.,1.],[1.,1.,0.]])
eigen_vector(A)

고윳값
[-1.00000000000000, 2.00000000000000]


[array([[-1.],
        [ 0.],
        [ 1.]]),
 array([[ 0.],
        [-1.],
        [ 1.]]),
 array([[1.],
        [1.],
        [1.]])]

#### 일부 행렬에 대해서는 오류 발생

##### 
##### 실제로는 넘파이 메소드로 매우 쉽게 구할 수 있다. 
##### 고유벡터의 경우 계산 방법에 의해 값이 다르게 나타날 수 있으며, 정규화하는 것이 일반적이다.

In [345]:
A = np.array([[0.,1.,1.],[1., 0.,1.],[1.,1.,0.]])
A = np.array([[1.,0.,0.,0.],[-3., -2.,0.,0.],[2.,3.,-1.,0.],[-2.,3.,0.,3.]])
np.linalg.eig(A)

(array([ 3., -1., -2.,  1.]),
 array([[ 0.        ,  0.        ,  0.        ,  0.34299717],
        [ 0.        ,  0.        ,  0.31068488, -0.34299717],
        [ 0.        ,  1.        , -0.93205465, -0.17149859],
        [ 1.        ,  0.        , -0.18641093,  0.85749293]]))

##### 행사다리꼴의 영이 아닌 행들은 선형독립이다.

#### 참고 자료
##### sympy 관련 : https://codepractice.tistory.com/73
##### 고유벡터 계산기
###### https://matrixcalc.org/ko/vectors.html#eigenvectors%28%7B%7B1,0,0,0%7D,%7B-3,-2,0,0%7D,%7B2,3,-1,0%7D,%7B-2,3,0,3%7D%7D%29
##### 고유벡터 관련 
###### https://m.blog.naver.com/PostView.nhn?blogId=seolgoons&logNo=221330201375&proxyReferer=http:%2F%2F203.233.19.219%2F
## 
## 
#### 이전 노트에서 정의한 함수

In [8]:
def permutation(array, k):
    order = [] #순서쌍을 담을 공간
    n = len(array)
    
    if k == 1:
        for element in array: #원소가 최종적으로 하나만 남는 경우
            order.append([element])

    elif k > 1:
        for i in range(n):
            A = array.copy()
            element = A.pop(i)


            for sub in permutation(A,k-1): #재귀함수의 결과를 순차적으로 추출
                #입력된 변수의 결과를 sub에 순차적으로 출력하나, 도중에 하위 재귀를 만났을때는 하위재귀부터 처리함(if k==1이 아닌 경우)
                order.append([element]+sub)    
    return  order

def det_cofactor(matrix): #1행전개로 구현
    
    det = 0 #결과를 담을 객체
    dim = len(matrix)
    
    if dim == 2: #2차 행렬일 때는 ad-bc 수행
        
        det = matrix[0][0]*matrix[1][1] - matrix[1][0]*matrix[0][1] 
        return det
    
    else : 
        
        for col in range(dim):
            #원소 a 추출
            a = matrix[0][col]

            #소행렬식 M 추출
            partial = np.delete(matrix,0,axis=0) #1행전개
            partial = np.delete(partial,col,axis=1)
            
            M = det_cofactor(partial) #재귀함수

            sgn = (-1)**(col)
            # 부호(실제 파이썬 인덱스는 0부터 시작하므로, 1행과 k열의 숫자 합은 col 변수와 동일한 홀/짝의 성질을 가짐)
            det += a*sgn*M

    return det